In [ ]:
#| default_exp core

# core

> Main functionality.

The main functionality is actually provided by the function below. `main` is just a *wrapper* that parses command-line arguments.

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import sys
import argparse
import pathlib
import datetime

import yaml
import pandas as pd

import sproc.extend
import sproc.hier
import sproc.assemble
import sproc.bundle
import sproc.postprocess
import sproc.structure
import sproc.download

Directory where the zip files are stored

In [ ]:
directory = pathlib.Path.cwd().parent / 'samples'
assert directory.exists()
directory

Path('/home/manu/Sync/UC3M/proyectos/2022/nextProcurement/sproc/samples')

## Processing a single zip file

In [ ]:
#| export
def cli_process_zip(args: list = None) -> None:
    
    parser = argparse.ArgumentParser(description='Process zip file')

    parser.add_argument('zip_file', type=argparse.FileType('r'), help='zip file')
    parser.add_argument('output_file', help='Output (parquet) file')

    command_line_arguments = parser.parse_args(args)
    
    output_file = pathlib.Path(command_line_arguments.output_file)
    assert output_file.suffix == '.parquet', 'a .parquet file was expected'
    
    data_df, deleted_series = sproc.assemble.distilled_data_from_zip(command_line_arguments.zip_file.name)
    
    res = sproc.assemble.merge_deleted(data_df, deleted_series)
    res = sproc.assemble.parquet_amenable(res)
    
    res.to_parquet(output_file)

In [ ]:
zip_file = directory /'yearly' / 'PlataformasAgregadasSinMenores_2018.zip'
assert zip_file.exists()
print(f'{zip_file=}')

zip_file=Path('/home/manu/Sync/UC3M/proyectos/2022/nextProcurement/sproc/samples/yearly/PlataformasAgregadasSinMenores_2018.zip')


In [ ]:
output_file = directory / 'year_2018.parquet'
print(f'{output_file=}')

output_file=Path('/home/manu/Sync/UC3M/proyectos/2022/nextProcurement/sproc/samples/year_2018.parquet')


In [ ]:
args = [zip_file.as_posix(), output_file.as_posix()]
cli_process_zip(args)

In [ ]:
%ls {directory}

2018-2021_20samples.parquet
extended_sample.parquet
gencat/
merged.parquet
PLACE.yaml
PlataformasAgregadasSinMenores_20220104_030016_1.atom
PlataformasAgregadasSinMenores_20220104_030016_1_single.atom
PlataformasAgregadasSinMenores_202201_05-06.zip
PlataformasAgregadasSinMenores_202201_08-11.zip
PlataformasAgregadasSinMenores_202201_28-29.zip
README.md
renamed_cols_extended_sample.parquet
year_2018.parquet
yearly/


In [ ]:
pd.read_parquet(output_file).head(2)

id  \
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
zip                                     file name                                          entry                                                      
PlataformasAgregadasSinMenores_2018.zip PlataformasAgregadasSinMenores_20180217_180137_... 453    https://contrataciondelestado.es/sindicacion/P...   
                                                                                           452    https://contrataciondelestado.es/sindicacion/P...   

                                                                                                                                            summary  \
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
zip                                     file name                                          entry                                                      
PlataformasAgregadasSinMenores_2018.zip PlataformasAgregadasSinMenores_20180217_180137_... 453    Expediente: 1284/17, Entidad: Diputación Provi...   
                                                                                           452    Expediente: 1282/17, Entidad: Diputación Provi...   

                                                                                                                                              title  \
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                  

## Extending historical data

A function to extend an existing *parquet* file with new data in a *zip* file.

In [ ]:
#| export
def cli_extend_parquet_with_zip(args: list = None) -> None:
    
    parser = argparse.ArgumentParser(description='Extend existing parquet file with data from a given zip')

    parser.add_argument('history_file', type=argparse.FileType('r'), help='Parquet file')
    parser.add_argument('zip_file', type=argparse.FileType('r'), help='Zip file')
    parser.add_argument('output_file', help='Output (parquet) file')

    command_line_arguments = parser.parse_args(args)
    
    history_file = pathlib.Path(command_line_arguments.history_file.name)
    zip_file = pathlib.Path(command_line_arguments.zip_file.name)
    
    output_file = pathlib.Path(command_line_arguments.output_file)
    assert output_file.suffix == '.parquet', 'a .parquet file was expected'
    
    sproc.extend.parquet_with_zip(history_file, zip_file, output_file)

Testing with some sample files

In [ ]:
history_file = directory /'2018-2021_20samples.parquet'
assert history_file.exists()
print(f'{history_file=}')

history_file=Path('/home/manu/Sync/UC3M/proyectos/2022/nextProcurement/sproc/samples/2018-2021_20samples.parquet')


In [ ]:
new_zip_file = directory / 'PlataformasAgregadasSinMenores_202201_28-29.zip'
assert new_zip_file.exists()
print(f'{new_zip_file=}')

new_zip_file=Path('/home/manu/Sync/UC3M/proyectos/2022/nextProcurement/sproc/samples/PlataformasAgregadasSinMenores_202201_28-29.zip')


In [ ]:
output_file = directory / 'extended_sample.parquet'
output_file

Path('/home/manu/Sync/UC3M/proyectos/2022/nextProcurement/sproc/samples/extended_sample.parquet')

In [ ]:
args = [history_file.as_posix(), new_zip_file.as_posix(), output_file.as_posix()]
cli_extend_parquet_with_zip(args)

In [ ]:
%ls {directory}

2018-2021_20samples.parquet
extended_sample.parquet
gencat/
merged.parquet
PLACE.yaml
PlataformasAgregadasSinMenores_20220104_030016_1.atom
PlataformasAgregadasSinMenores_20220104_030016_1_single.atom
PlataformasAgregadasSinMenores_202201_05-06.zip
PlataformasAgregadasSinMenores_202201_08-11.zip
PlataformasAgregadasSinMenores_202201_28-29.zip
README.md
renamed_cols_extended_sample.parquet
year_2018.parquet
yearly/


## Renaming columns

In [ ]:
#| export
def cli_rename_columns(args: list = None) -> None:
    
    parser = argparse.ArgumentParser(description='Rename columns')

    parser.add_argument('hierarchical_file', type=argparse.FileType('r'), help='(Hierarchical) Parquet file')
    parser.add_argument('mapping_file', type=argparse.FileType('r'), help='YAML file mapping hierarchical colum names to plain ones')
    parser.add_argument('output_file', help='Output (parquet) file')

    command_line_arguments = parser.parse_args(args)
    
    hierarchical_file = pathlib.Path(command_line_arguments.hierarchical_file.name)
    assert hierarchical_file.suffix == '.parquet', 'a (hierarchical) .parquet file was expected'
    
    mapping_file = pathlib.Path(command_line_arguments.mapping_file.name)
    assert (mapping_file.suffix == '.yaml') or (mapping_file.suffix == '.YAML'), 'a YAML file was expected'
    
    output_file = pathlib.Path(command_line_arguments.output_file)
    assert output_file.suffix == '.parquet', 'a .parquet file was expected'
    
    with mapping_file.open() as yaml_data:
        data_scheme = yaml.load(yaml_data, Loader=yaml.FullLoader)
        
    df = pd.read_parquet(hierarchical_file)
    renamed_cols_df = sproc.hier.flatten_columns_names(df, data_scheme)
    
    renamed_cols_df.to_parquet(output_file)

In [ ]:
renamed_cols_output_file = directory / 'renamed_cols_extended_sample.parquet'
renamed_cols_output_file

Path('/home/manu/Sync/UC3M/proyectos/2022/nextProcurement/sproc/samples/renamed_cols_extended_sample.parquet')

In [ ]:
mapping_file = directory / 'PLACE.yaml'
assert mapping_file.exists()
mapping_file

Path('/home/manu/Sync/UC3M/proyectos/2022/nextProcurement/sproc/samples/PLACE.yaml')

In [ ]:
args = [output_file.as_posix(), mapping_file.as_posix(), renamed_cols_output_file.as_posix()]
cli_rename_columns(args)

In [ ]:
pd.read_parquet(renamed_cols_output_file).head(2)

id  \
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20180217_180137_... 453    https://contrataciondelestado.es/sindicacion/P...   
                                                   452    https://contrataciondelestado.es/sindicacion/P...   

                                                               zip  \
file name                                          entry             
PlataformasAgregadasSinMenores_20180217_180137_... 453    some.zip   
                                                   452    some.zip   

                                                                                                    summary  \
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20180217_180137_... 453    Expediente: 1284/17, Entidad: Diputación Provi...   
                                                   452    Expediente: 1282/17, Entidad: Diputación Provi...   

                                                                                                      title  \
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20180217_180137_... 453    Refuerzo de Firme en la VP 3001 Renedo de Esgu...   
                                                   452    Refuerzo de Firme en la VP 6603 Mota del Marqu...   

                                                                                  updated  \
file name                                          entry                                    
PlataformasAgregadasSinMenores_20180217_180137_... 453   2018-01-02 08:01:52.024000+00:00   
                                                   452   2018-01-02 08:02:24.833000+00:00   

                                                         Número de Expediente  \
file name                                          entry                        
PlataformasAgregadasSinMenores_20180217_180137_... 453                1284/17   
                                                   452                1282/17   

                                                         Estado         ID  \
file name                                          entry                     
PlataformasAgregadasSinMenores_20180217_180137_... 453      RES  L02000047   
                                                   452      RES  L02000047   

                                                                                       Nombre  \
file name                                          entry                                        
PlataformasAgregadasSinMenores_20180217_180137_... 453    Diputación Provincial de Valladolid   
                                                   452    Diputación Provincial de Valladolid   

                                                                                        Objeto del Contrato  \
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20180217_180137_... 453    Refuerzo de Firme en la VP 3001 Renedo de Esgu...   
                                                   452    Refuerzo de Firme en la VP 6603 Mota del Marqu...   

                                                          ...  \
file name                                          entry  ...   
PlataformasAgregadasSinMenores_20180217_180137_... 453    ...   
                                                   452    ...   

                                                          Pliego de cláusulas administrativas (URI)  \
file name                                          entry                                              
PlataformasAgregadasSinMenores_20180217_180137_... 453                                         <NA>   
                                                   452                    

## Reading a bunch of zip files

It receives a `list` of *zip* files and returns a (column-hierarchical) `pd.DataFrame` encompassing all the data

In [ ]:
#| export
def read_zips(
    files: list[str | pathlib.Path] # Input files
    ) -> pd.DataFrame: # Procurement data
    "Build a `DataFrame` out of a bunch of zip files"
    
    # at the beginning it is guaranteed that every file is present
    for f in files:
        
        # in case `str` (rather than `Pathlib`s) were passed
        f = pathlib.Path(f)
        
        assert f.exists(), f'{f} doesn\'t exist'
    
    # accumulators for the data itself (contracts) and records of deleted entries
    res_df = None
    res_deleted_series = None

    for f in files:

        print(f'Processing "{f}"')

        # data is read from the above *zip* file, and `concatenate`d into a single `pd.DataFrame`...
        df = sproc.bundle.read_zip(f, concatenate=True)

        # ...which is re-structured with multiindexed columns
        df = sproc.hier.flat_df_to_multiindexed_df(df)

        # every ATOM inside the zip file also contains information (at the beginning) about deleted entries
        deleted_series = sproc.bundle.read_deleted_zip(f)

        # if this is NOT the first iteration...
        if res_df is not None:

            # ...the new data is stacked
            res_df = sproc.assemble.stack(res_df, df)
            res_deleted_series = pd.concat((res_deleted_series, deleted_series), axis=0)

        # ...if this is the first iteration
        else:

            # ...the new data is set as the accumulated result
            res_df = df
            res_deleted_series = deleted_series
            
    # some contracts show up more than once, and only the last update is to be kept
    res_last_update_only_df = sproc.postprocess.keep_updates_only(res_df)

    # a new *deleted* `pd.Series` is built by dropping duplicates (again, only the last one is kept)
    deduplicated_deleted_series = sproc.postprocess.deduplicate_deleted_series(res_deleted_series)

    # the *deleted* series is used to flag the appropriate entries in the "main" `pd.DataFrame`;
    # the result is "stateful" in the sense that we know the state of each entry (deleted -and, if so, when- or not)
    stateful_df = sproc.assemble.merge_deleted(res_last_update_only_df, deduplicated_deleted_series)
    
    # the number of filled-in rows for column `deleted_on` should match the number of `id`s in `deduplicated_deleted_series` that show up in `stateful_df`
    assert stateful_df['deleted_on'].notna().sum() == len(set(stateful_df['id']) & set(deduplicated_deleted_series.index.get_level_values(2)))
            
    return stateful_df

Let us pick a couple of files for testing

In [ ]:
zip_files = ['PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_2019.zip']
zip_files = [directory/ 'yearly' / e for e in zip_files]
zip_files

[Path('/home/manu/Sync/UC3M/proyectos/2022/nextProcurement/sproc/samples/yearly/PlataformasAgregadasSinMenores_2018.zip'),
 Path('/home/manu/Sync/UC3M/proyectos/2022/nextProcurement/sproc/samples/yearly/PlataformasAgregadasSinMenores_2019.zip')]

In [ ]:
df = read_zips(zip_files)
df.head()

Processing "/home/manu/Sync/UC3M/proyectos/2022/nextProcurement/sproc/samples/yearly/PlataformasAgregadasSinMenores_2018.zip"
Processing "/home/manu/Sync/UC3M/proyectos/2022/nextProcurement/sproc/samples/yearly/PlataformasAgregadasSinMenores_2019.zip"


id  \
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
zip                                     file name                                          entry                                                      
PlataformasAgregadasSinMenores_2018.zip PlataformasAgregadasSinMenores_20180217_180137_... 453    https://contrataciondelestado.es/sindicacion/P...   
                                                                                           452    https://contrataciondelestado.es/sindicacion/P...   
                                                                                           451    https://contrataciondelestado.es/sindicacion/P...   
                                                                                           450    https://contrataciondelestado.es/sindicacion/P...   
                                                                                           449    https://contrataciondelestado.es/sindicacion/P...   

                                                                                                                                            summary  \
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
zip                                     file name                                          entry                                                      
PlataformasAgregadasSinMenores_2018.zip PlataformasAgregadasSinMenores_20180217_180137_... 453    Expediente: 1284/17, Entidad: Diputación Provi...   
                                                                                           452    Expediente: 1282/17, Entidad: Diputación Provi...   
                                                                                           451    Expediente: 1281/17, Entidad: Diputación Provi...   
                                                                                           450    Id licitación: VI/17/04-015; Órgano de Contrat...   
                                                                   

### CLI

A companion function to allow using the above from the command-line.

In [ ]:
#| export
def cli_read_zips(args: list = None) -> None:
    
    parser = argparse.ArgumentParser(description='Process a bunch of zip files')

    parser.add_argument('input_files', type=argparse.FileType('r'), nargs='+', help='zip files')
    parser.add_argument('-o', '--output_file', default='out.parquet', help='Output (parquet) file')

    command_line_arguments = parser.parse_args(args)
    
    output_file = pathlib.Path(command_line_arguments.output_file)
    assert output_file.suffix == '.parquet', 'a .parquet file was expected'
        
    # the `pd.DataFrame` is built...
    df = read_zips([f.name for f in command_line_arguments.input_files])
    
    # ...rearranged for saving in parquet format
    parquet_df = sproc.assemble.parquet_amenable(df)
    
    parquet_df.to_parquet(output_file)
    
    print(f'writing {output_file}...')

In [ ]:
cli_read_zips([e.as_posix() for e in zip_files] + '-o o.parquet'.split())

Processing "/home/manu/Sync/UC3M/proyectos/2022/nextProcurement/sproc/samples/yearly/PlataformasAgregadasSinMenores_2018.zip"
Processing "/home/manu/Sync/UC3M/proyectos/2022/nextProcurement/sproc/samples/yearly/PlataformasAgregadasSinMenores_2019.zip"
writing o.parquet...


## Updates

In [ ]:
datetime.datetime.today()

datetime.datetime(2023, 1, 2, 19, 5, 16, 588473)

In [ ]:
def update(
    kind: str, # One of 'outsiders', 'insiders', or 'minors'
    output_directory: str | pathlib.Path # The path where hosting
    ):

    # `kind` should be one of the pre-set types
    assert kind in sproc.structure.tables

    # just in case
    output_directory = pathlib.Path(output_directory)

    # output_file = pathlib.Path(kind).with_suffix('.parquet')

    df = pd.read_parquet(output_file)
    from_date = df['updated'].max().to_pydatetime() - datetime.timedelta(days=1)
    # print(f'from: {from_date}')
    # print(f'updated: {df["updated"].max().to_pydatetime() }')

    downloaded_files = sproc.download.from_date(kind, date=from_date, output_directory=output_directory)

Let us make a new directory...

In [ ]:
output_directory = pathlib.Path.cwd().parent / 'downloads'
output_directory.mkdir(exist_ok=True)
print(output_directory)

/home/manu/Sync/UC3M/proyectos/2022/nextProcurement/sproc/downloads


In [ ]:
update('outsiders', output_directory)

Output()

"PlataformasAgregadasSinMenores_202202.zip" already exists

"PlataformasAgregadasSinMenores_202203.zip" already exists

"PlataformasAgregadasSinMenores_202204.zip" already exists

"PlataformasAgregadasSinMenores_202205.zip" already exists

...

"PlataformasAgregadasSinMenores_202208.zip" already exists

"PlataformasAgregadasSinMenores_202209.zip" already exists

"PlataformasAgregadasSinMenores_202210.zip" already exists

"PlataformasAgregadasSinMenores_202211.zip" already exists

"PlataformasAgregadasSinMenores_202212.zip" already exists

In [ ]:
df = pd.read_parquet(output_directory / 'outsiders.parquet')
df.head()

id  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20211101_030029.... 465    https://contrataciondelestado.es/sindicacion/P...   
                                                   462    https://contrataciondelestado.es/sindicacion/P...   
                                                   458    https://contrataciondelestado.es/sindicacion/P...   
                                                   456    https://contrataciondelestado.es/sindicacion/P...   
                                                   430    https://contrataciondelestado.es/sindicacion/P...   

                                                                                                    summary  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20211101_030029.... 465    Id licitación: 2021/725; Órgano de Contratació...   
                                                   462    Id licitación: 2021-000265; Órgano de Contrata...   
                                                   458    Id licitación: SUB_OBE_2021_0014; Órgano de Co...   
                                                   456    Id licitación: EC21/000111; Órgano de Contrata...   
                                                   430    Id licitación: 3455/2021; Órgano de Contratac...   

                                                                                                      title  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20211101_030029.... 465    Subministrament d'aigua mineral amb cinc fonts...   
                                                   462    Servei de  publicació i seguiment de contingut...   
                                                   458    L'objecte del contracte és contractar una empr...

In [ ]:
# df['updated'].max().to_pydatetime() - datetime.timedelta(days=1)
df['updated'].max().to_pydatetime()

datetime.datetime(2022, 1, 1, 0, 0, 17, 920000, tzinfo=<UTC>)

In [ ]:
#| hide
from nbdev.doclinks import nbdev_export

In [ ]:
#| hide
nbdev_export('00_core.ipynb')